# Giotto-tda

Install packages and load libraries

In [ ]:
import numpy as np
import pandas as pd

from keras.datasets import mnist

import matplotlib.pyplot as plt
# %matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline, make_union

from gtda.plotting import plot_heatmap
from gtda.images import Binarizer, RadialFiltration, HeightFiltration
from gtda.homology import CubicalPersistence
from gtda.diagrams import Scaler, HeatKernel, Amplitude, PersistenceEntropy

Import MNIST data from Keras

In [ ]:
# import mnist data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(train_images.ndim, train_images.shape)

11501568/11490434 [==============================] - 0s 0us/step
3 (60000, 28, 28)


Reshape individual image and plot on heatmap

In [ ]:
# Reshape image to (28,28) and plot on heatmap
imInd = 1
im = test_images[imInd].reshape(28,28)
plot_heatmap(im) #heatmap of image

Binarized image

In [ ]:
binarizer = Binarizer(threshold=0.4)
im_binarized = binarizer.fit_transform(im)
im_binarized_num = 1*im_binarized

plot_heatmap(im_binarized_num) #heatmap of binarized image

Radial filtration

In [ ]:
radial_filtration = RadialFiltration(center=np.array([20, 6]))
im_filtration = radial_filtration.fit_transform(im_binarized.reshape(-1,28,28))

radial_filtration.plot(im_filtration, colorscale="jet") #heatmap with radial filtration

Persistence Diagram

In [ ]:
cubical_persistence = CubicalPersistence(n_jobs=-1)
im_cubical = cubical_persistence.fit_transform(im_filtration)

cubical_persistence.plot(im_cubical) #persistence diagram

Rescaled Persistence Diagram

In [ ]:
scaler = Scaler()
im_scaled = scaler.fit_transform(im_cubical)

scaler.plot(im_scaled) #scaled diagram

Representation

In [ ]:
heat = HeatKernel(sigma=.15, n_bins=60, n_jobs=-1)
im_heat = heat.fit_transform(im_scaled)

# Visualise the heat kernel for H1
heat.plot(im_heat, homology_dimension_idx=1, colorscale='jet') #heatmap for H1

Pipline

In [ ]:
steps = [
    ("binarizer", Binarizer(threshold=0.4)),
    ("filtration", RadialFiltration(center=np.array([20, 6]))),
    ("diagram", CubicalPersistence()),
    ("rescaling", Scaler()),
    ("amplitude", Amplitude(metric="heat", metric_params={'sigma':0.15, 'n_bins':60}))
]

heat_pipeline = Pipeline(steps)

In [ ]:
im_pipeline = heat_pipeline.fit_transform(im.reshape((-1,28,28)))
im_pipeline

array([[0.        , 1.49944247]])

In [ ]:
direction_list = [[1, 0], [1, 1], [0, 1], [-1, 1], [-1, 0], [-1, -1], [0, -1], [1, -1]]

center_list = [
    [13, 6],
    [6, 13],
    [13, 13],
    [20, 13],
    [13, 20],
    [6, 6],
    [6, 20],
    [20, 6],
    [20, 20],
]

# Creating a list of all filtration transformer, we will be applying
filtration_list = (
    [
        HeightFiltration(direction=np.array(direction), n_jobs=-1)
        for direction in direction_list
    ]
    + [RadialFiltration(center=np.array(center), n_jobs=-1) for center in center_list]
)

# Creating the diagram generation pipeline
diagram_steps = [
    [
        Binarizer(threshold=0.4, n_jobs=-1),
        filtration,
        CubicalPersistence(n_jobs=-1),
        Scaler(n_jobs=-1),
    ]
    for filtration in filtration_list
]

# Listing all metrics we want to use to extract diagram amplitudes
metric_list = [
    {"metric": "bottleneck", "metric_params": {}},
    {"metric": "wasserstein", "metric_params": {"p": 1}},
    {"metric": "wasserstein", "metric_params": {"p": 2}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 2, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 2, "n_bins": 100}},
    {"metric": "betti", "metric_params": {"p": 1, "n_bins": 100}},
    {"metric": "betti", "metric_params": {"p": 2, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 1.6, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 1, "sigma": 3.2, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 1.6, "n_bins": 100}},
    {"metric": "heat", "metric_params": {"p": 2, "sigma": 3.2, "n_bins": 100}},
]

#
feature_union = make_union(
    *[PersistenceEntropy(nan_fill_value=-1)]
    + [Amplitude(**metric, n_jobs=-1) for metric in metric_list]
)

tda_union = make_union(
    *[make_pipeline(*diagram_step, feature_union) for diagram_step in diagram_steps],
    n_jobs=-1
)

In [ ]:
#reduces train and test set size
train_size, test_size = 60, 20

X_train = train_images[0:train_size]
y_train = train_labels[0:train_size]
X_test = test_images[0:test_size]
y_test = test_labels[0:test_size]

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train_tda = tda_union.fit_transform(X_train)
print(X_train_tda.shape)

X_train shape: (60, 28, 28), y_train shape: (60,)
X_test shape: (20, 28, 28), y_test shape: (20,)
(60, 476)


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train_tda, y_train)

X_test_tda = tda_union.transform(X_test)
rf.score(X_test_tda, y_test)

0.8